# 5章 誤差逆伝播法

- 重みパラメータに関する損失関数の勾配は数値微分によって求めた
- 実装は簡単だけど、時間がかかる難点がある
- 重みパラメータの勾配計算を効率良く行う「誤差逆伝播法」を学ぶ
- 誤差逆伝播法の理解には数式と計算グラフが良さそう（筆者的に）
- この章では計算グラフで説明する
- 計算グラフによって説明するアイデアは↓を参考にしてるらしい
- http://karpathy.github.io/neuralnets/
- http://cs231n.stanford.edu/


> ここで、たとえば、リンゴの値段が値上がりした場合、最終的な支払金額にどのように影響するかを知りたいとしましょう。これは、「リンゴの値段に関する支払金額の微分」を求めることに相当します。

In [7]:
from sympy import *

z = Function("z")
t = Symbol("t")
x = Symbol("x")
y = Symbol("y")
z = (x + y) ** 2
t = (x + y)


In [8]:
diff(z, x)

2*x + 2*y

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy


In [10]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

# backward

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)


220.00000000000003
2.2 110.00000000000001 200


次回

## 5.4.2 加算レイヤの実装

デュエルスタンバイ

In [1]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    


In [3]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1


# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) 
orange_price = mul_orange_layer.forward(orange, orange_num) 
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [4]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx


In [5]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [6]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx


次回

## 5.6 Affine / Softmaxレイヤの実装

デュエルスタンバイ